In [1]:
import pandas as pd
import numpy as np
import tqdm

In [2]:
#last, first, min, max,std,mean,median
importnant_columns = ['customer_ID', 'S_2',
                      'P_2', 'P_3',
                      'D_39', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48','D_49', 'D_62', 'D_75', 'D_104', 'D_141',
                      'B_2', 'B_3', 'B_4', 'B_8', 'B_9', 'B_18', 'B_11', 'B_23', 'B_33',
                      'R_1', 'R_27',
                      'S_3', 'S_25',
                     "B_30","B_38","D_114","D_117","D_120","D_64","D_68"]
                      #'target']

In [3]:
train = pd.read_pickle(r'C:\Users\domin\Downloads\AE_pickled\test.pkl')[importnant_columns]
stat_list = ['first', 'last', 'median', 'mean', 'std', 'max']
#test = pd.read_pickle(r'C:\Users\domin\Downloads\AE_pickled\test.pkl')

In [4]:
def number_cat(list_of_categories):
    """Function to cast categorical vatriables as numeric, output - dictionary of mappings"""
    unique_values = list_of_categories.values.unique()
    corresponding_map = {}
    i = 0
    for q in unique_values:
        corresponding_map[q] = i
        i +=1
    return corresponding_map


In [5]:
mapping = number_cat(train['D_64'])
train['D_64'] = [mapping[q] for q in train['D_64']]

In [6]:
def process_and_feature_engineer(df):
    # FEATURE ENGINEERING FROM 
    # https://www.kaggle.com/code/huseyincot/amex-agg-data-how-it-created
    all_cols = [c for c in list(df.columns) if c not in ['customer_ID','S_2']]
    cat_features = ["B_30","B_38","D_114","D_117","D_120","D_64","D_68"]
    num_features = [x for x in all_cols if x not in cat_features]

    test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'sem', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]

    test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['first', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    df = pd.concat([test_num_agg, test_cat_agg], axis=1)
    del test_num_agg, test_cat_agg
    print('shape after engineering', df.shape )
    print(df.columns)
    return df
train_processed = process_and_feature_engineer(train)

shape after engineering (924621, 161)
Index(['P_2_mean', 'P_2_sem', 'P_2_min', 'P_2_max', 'P_2_last', 'P_3_mean',
       'P_3_sem', 'P_3_min', 'P_3_max', 'P_3_last',
       ...
       'D_117_nunique', 'D_120_first', 'D_120_last', 'D_120_nunique',
       'D_64_first', 'D_64_last', 'D_64_nunique', 'D_68_first', 'D_68_last',
       'D_68_nunique'],
      dtype='object', length=161)


In [7]:
targets = train[['customer_ID', 'target']].drop_duplicates().set_index('customer_ID')
targets

,target
customer_ID,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0
...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,0
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,0


In [8]:
train_processed

,P_2_mean,P_2_sem,P_2_min,P_2_max,P_2_last,P_3_mean,P_3_sem,P_3_min,P_3_max,P_3_last,...,D_117_nunique,D_120_first,D_120_last,D_120_nunique,D_64_first,D_64_last,D_64_nunique,D_68_first,D_68_last,D_68_nunique
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.933594,0.006710,0.868652,0.960449,0.934570,0.680176,0.014058,0.581543,0.741699,0.629395,...,1,0.0,0.0,1,0,0,1,6.0,6.0,1
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.899902,0.006129,0.861328,0.929199,0.880371,0.566895,0.010224,0.510254,0.619141,0.570801,...,1,0.0,0.0,2,0,0,1,6.0,6.0,1
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.878418,0.007998,0.797852,0.904297,0.880859,0.618164,0.020972,0.381104,0.678711,0.628906,...,1,0.0,0.0,1,1,1,1,6.0,6.0,1
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.599121,0.005570,0.567383,0.623535,0.621582,0.610840,0.024979,0.345215,0.704102,0.671875,...,2,0.0,0.0,1,0,0,1,2.0,3.0,3
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.891602,0.011736,0.805176,0.940430,0.872070,0.527344,0.024542,0.254395,0.584473,0.570312,...,1,0.0,0.0,1,0,0,1,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,0.848633,0.011636,0.730469,0.895508,0.844238,0.696289,0.036490,0.363281,0.814453,0.531250,...,2,0.0,1.0,2,3,3,1,6.0,6.0,1
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0.859375,0.003456,0.831055,0.868164,0.831055,0.606934,0.008335,0.541992,0.649414,0.562500,...,2,0.0,0.0,1,1,1,1,6.0,6.0,1
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,0.786621,0.003964,0.756836,0.802734,0.800293,0.633301,0.010275,0.572754,0.682617,0.616211,...,1,0.0,0.0,1,3,3,1,5.0,5.0,1


In [9]:
train_processed = train_processed.merge(targets, how = 'left', left_index = True, right_index = True)

In [7]:
train_processed

,P_2_mean,P_2_sem,P_2_min,P_2_max,P_2_last,P_3_mean,P_3_sem,P_3_min,P_3_max,P_3_last,...,D_117_nunique,D_120_first,D_120_last,D_120_nunique,D_64_first,D_64_last,D_64_nunique,D_68_first,D_68_last,D_68_nunique
customer_ID,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.601562,0.006733,0.568848,0.631348,0.568848,0.591797,0.019578,0.535156,0.634277,0.560547,...,1,1.0,1.0,1,0,1,2,6.0,6.0,1
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.862305,0.008718,0.794434,0.913574,0.841309,0.553223,0.022999,0.385742,0.653809,0.525879,...,1,0.0,0.0,1,2,2,1,6.0,6.0,1
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.749023,0.017039,0.673340,0.834961,0.697754,0.670898,0.053012,0.381592,0.944336,0.566406,...,2,0.0,0.0,1,2,1,2,6.0,4.0,2
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.474609,0.008011,0.428467,0.514160,0.513184,0.610840,0.024277,0.464844,0.794434,0.628418,...,3,0.0,0.0,2,3,3,1,5.0,5.0,1
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.323975,0.013836,0.254395,0.425781,0.254395,0.634766,0.034989,0.467529,0.877441,0.569824,...,1,0.0,1.0,2,2,3,2,6.0,5.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0.631836,0.004373,0.597656,0.646973,0.646973,0.629395,0.013860,0.556641,0.757812,0.662109,...,1,0.0,0.0,1,2,2,1,6.0,6.0,1
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0.546875,0.014773,0.471191,0.634766,0.471191,0.599121,0.032992,0.443359,0.846191,0.479004,...,1,1.0,0.0,2,2,2,1,5.0,6.0,2
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,0.239014,0.013424,0.195679,0.316650,0.206421,0.583496,0.032476,0.405273,0.729980,0.598633,...,1,1.0,0.0,2,2,2,1,6.0,6.0,1


In [8]:
len(train_processed)

924621

In [9]:
train_processed.to_pickle('test_v2.pkl')